In [4]:
!pip install src

  Using cached src-0.0.7.zip (6.3 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for src
  Running setup.py clean for src
Failed to build src
ERROR: Could not build wheels for src, which is required to install pyproject.toml-based projects


In [6]:
import json
import time
from io import StringIO
import networkx as nx
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# from resources.handlers.zos.zosmanager import ZosManager
from src.PurchaseAnalytics.reas.searchable_resume_app.utils_common import constants


def _add_node_and_edge(
        graph: nx.DiGraph,
        candidate_node: str,
        node_for_adding: str,
        node_type: str,
        candidate_to_node_relation: str,
        node_to_candidate_relation: str,
        predicted_value="",
        actual_value=""
):
    """
    Creates nodes and edges for the knowledge graph

    Parameters
    ----------
    graph: nx.DiGraph
        Contains the existing knowledge graph of resumes
    candidate_node: str
        Name of the candidate
    node_for_adding: str
        Name of node to be added
    node_type: str
        Type of node to be added
    candidate_to_node_relation: str
        Edge relation between candidate and node to be added
    node_to_candidate_relation: str
        Edge relation between node to be added and candidate

    Returns
    ----------
    graph: nx.DiGraph
        Updated knowledge graph of resumes
    """
    if not node_for_adding:
        return graph

    try:

        actual_name = node_for_adding
        if isinstance(node_for_adding, str):
            node_for_adding = node_for_adding.lower()

        if not graph.has_node(node_for_adding):
            graph.add_node(node_for_adding=node_for_adding, node_type=node_type, actual_name=actual_name,
                           predicted_value=predicted_value, actual_value=actual_value)

        graph.add_edge(u_of_edge=node_for_adding, v_of_edge=candidate_node, relation=node_to_candidate_relation)
        if candidate_to_node_relation:
            graph.add_edge(u_of_edge=candidate_node, v_of_edge=node_for_adding,
                           relation=candidate_to_node_relation)
        return graph

    except Exception as e:
        print(f'Skipping node creation for node: {node_for_adding}\n{e}')
        return graph


def construct_graph(df):
    """
    Constructs a knowledge graph from all the resumes.

    Parameters
    ----------
    df: pd.DataFrame
        Contains resume data of all candidates in a data frame

    Returns
    ---------
    graph: nx.DiGraph()
        Knowledge graph for the resumes
    """

    start_time = time.time()
    df = df.replace(np.nan, None)
    df['resume_id'] = [i for i in range(0, len(df))]
    print("typeee", df['predictions_primary_information_professional_information_current_designation'].dtype)
    graph = nx.DiGraph()
    column_name = 'predictions_primary_information_professional_information_current_designation'
    df[column_name] = df[column_name].apply(lambda x: "[]" if pd.isna(x) else x)
    column_name_2 = "predictions_primary_information_contact_information_emails"
    df[column_name_2] = df[column_name_2].apply(lambda x: "[]" if pd.isna(x) else x)
    for _, row in df.iterrows():
        unique_id_node = row['resume_id']

        # Add candidate name
        graph.add_node(node_for_adding=unique_id_node, node_type='RESUME_ID')

        # Add candidate name as nodes
        if row['predictions_primary_information_candidate_name']:
            graph = _add_node_and_edge(
                graph=graph,
                candidate_node=unique_id_node,
                node_for_adding=row['predictions_primary_information_candidate_name'],
                node_type='CANDIDATE_NAME',
                candidate_to_node_relation='CANDIDATE_NAME',
                node_to_candidate_relation='CANDIDATE_NAME_OF'
            )

        if row['predictions_primary_information_demographics_age']:
            graph = _add_node_and_edge(
                graph=graph,
                candidate_node=unique_id_node,
                node_for_adding=row['predictions_primary_information_demographics_age'],
                node_type='AGE',
                candidate_to_node_relation='AGE',
                node_to_candidate_relation='AGE_OF'
            )

        if row['predictions_primary_information_demographics_date_of_birth']:
            graph = _add_node_and_edge(
                graph=graph,
                candidate_node=unique_id_node,
                node_for_adding=row['predictions_primary_information_demographics_date_of_birth'],
                node_type='DATE_OF_BIRTH',
                candidate_to_node_relation='DATE_OF_BIRTH',
                node_to_candidate_relation='DATE_OF_BIRTH_OF'
            )

        if row['predictions_primary_information_demographics_gender']:
            graph = _add_node_and_edge(
                graph=graph,
                candidate_node=unique_id_node,
                node_for_adding=row['predictions_primary_information_demographics_gender'],
                node_type='GENDER',
                candidate_to_node_relation='GENDER',
                node_to_candidate_relation='GENDER_OF'
            )

        if row['predictions_primary_information_demographics_marital_status']:
            graph = _add_node_and_edge(
                graph=graph,
                candidate_node=unique_id_node,
                node_for_adding=row['predictions_primary_information_demographics_marital_status'],
                node_type='MARITAL_STATUS',
                candidate_to_node_relation='MARITAL_STATUS',
                node_to_candidate_relation='MARITAL_STATUS_OF'
            )

        if row['predictions_primary_information_demographics_nationalities']:
            graph = _add_node_and_edge(
                graph=graph,
                candidate_node=unique_id_node,
                node_for_adding=row['predictions_primary_information_demographics_nationalities'],
                node_type='NATIONALITY',
                candidate_to_node_relation='NATIONALITY',
                node_to_candidate_relation='NATIONALITY_OF'
            )

        if row['predictions_primary_information_links_linkedin']:
            graph = _add_node_and_edge(
                graph=graph,
                candidate_node=unique_id_node,
                node_for_adding=row['predictions_primary_information_links_linkedin'],
                node_type='LINKEDIN',
                candidate_to_node_relation='LINKEDIN',
                node_to_candidate_relation='LINKEDIN_OF'
            )

        if row['predictions_primary_information_links_repositories_github']:
            graph = _add_node_and_edge(
                graph=graph,
                candidate_node=unique_id_node,
                node_for_adding=row['predictions_primary_information_links_repositories_github'],
                node_type='GITHUB',
                candidate_to_node_relation='GITHUB',
                node_to_candidate_relation='GITHUB_OF'
            )

        for current_role in eval(row['predictions_primary_information_professional_information_current_designation']):
            if current_role:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=current_role,
                    node_type='CURRENT ROLE',
                    candidate_to_node_relation='CURRENTLY_WORKING_AS',
                    node_to_candidate_relation='CURRENT_ROLE_FOR'
                )

        if row['predictions_primary_information_professional_information_current_employer']:
            graph = _add_node_and_edge(
                graph=graph,
                candidate_node=unique_id_node,
                node_for_adding=row['predictions_primary_information_professional_information_current_employer'],
                node_type='CURRENT_EMPLOYER',
                candidate_to_node_relation='CURRENT_EMPLOYER',
                node_to_candidate_relation='CURRENT_EMPLOYER_OF'
            )

        if row['predictions_primary_information_professional_information_total_duration_of_experience_years']:
            graph = _add_node_and_edge(
                graph=graph,
                candidate_node=unique_id_node,
                node_for_adding=row[
                    'predictions_primary_information_professional_information_total_duration_of_experience_years'],
                node_type='YEARS_OF_EXPERIENCE',
                candidate_to_node_relation='YEARS_OF_EXPERIENCE',
                node_to_candidate_relation='YEARS_OF_EXPERIENCE_OF'
            )

        for i in range(0, 6):
            if row[f'predictions_education_{i}_board_of_study_actual_value']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[f'predictions_education_{i}_board_of_study_actual_value'],
                    node_type='BOARD_OF_STUDY',
                    candidate_to_node_relation='BOARD_OF_STUDY',
                    node_to_candidate_relation='BOARD_OF_STUDY_OF',
                    predicted_value=row[f'predictions_education_{i}_board_of_study_predicted_label']
                )

        for i in range(0, 6):
            if row[f'predictions_education_{i}_course_actual_value']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[f'predictions_education_{i}_course_actual_value'],
                    node_type='EDUCATIONAL_COURSE',
                    candidate_to_node_relation=f'STUDIED_COURSE_IN_INSTITUTE_{i}',
                    node_to_candidate_relation='STUDIED_BY',
                    predicted_value=row[f'predictions_education_{i}_course_predicted_label']
                )

        for i in range(0, 6):
            if row[f'predictions_education_{i}_course_field_of_study_category']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[f'predictions_education_{i}_course_field_of_study_category'],
                    node_type='COURSE_FIELD_OF_STUDY_CATEGORY',
                    candidate_to_node_relation=f'COURSE_FIELD_OF_STUDY_CATEGORY_{i}',
                    node_to_candidate_relation='COURSE_FIELD_OF_STUDY_CATEGORY_BY',
                    predicted_value=row[f'predictions_education_{i}_course_predicted_label']
                )

        for i in range(0, 6):
            if row[f'predictions_education_{i}_qualification_predicted_label']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[f'predictions_education_{i}_course_predicted_label'],
                    node_type='EDUCATIONAL QUALIFICATION',
                    candidate_to_node_relation=f'HAS_QUALIFICATION_FROM_INSTITUTE_{i}',
                    node_to_candidate_relation=f'COMPLETED_BY',
                    predicted_value=row[f'predictions_education_{i}_course_predicted_label'],
                    actual_value=row[f'predictions_education_{i}_qualification_actual_value']
                )

        for i in range(0, 6):
            if row[f'predictions_education_{i}_institute_institute_name']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[f'predictions_education_{i}_institute_institute_name'],
                    node_type='EDUCATIONAL INSTITUTE',
                    candidate_to_node_relation=f'STUDIED_AT_INSTITUTE_{i}',
                    node_to_candidate_relation='ALMA_MATTER_OF',
                    predicted_value=row[f'predictions_education_{i}_course_predicted_label']
                )

        # Add candidate phone number as nodes
        for i in range(0, 1):
            if row[f'predictions_primary_information_contact_information_phone_numbers_{i}_phone_number']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[
                        f'predictions_primary_information_contact_information_phone_numbers_{i}_phone_number'],
                    node_type='PHONE_NUMBER',
                    candidate_to_node_relation='PHONE_NUMBER',
                    node_to_candidate_relation='PHONE_NUMBER_OF'
                )

        # Add candidate email as nodes
        for email in eval(row['predictions_primary_information_contact_information_emails']):
            if email and email is not None:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=email,
                    node_type='EMAIL',
                    candidate_to_node_relation='EMAIL',
                    node_to_candidate_relation='EMAIL_OF'
                )

        if row['filename']:
            graph = _add_node_and_edge(
                graph=graph,
                candidate_node=unique_id_node,
                node_for_adding=row['filename'],
                node_type='FILE_NAME',
                candidate_to_node_relation='RESUME',
                node_to_candidate_relation='RESUME_OF'
            )

        for i in range(0, 2):
            if row[f'predictions_primary_information_contact_information_addresses_{i}_address_line']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[
                        f'predictions_primary_information_contact_information_addresses_{i}_address_line'],
                    node_type='ADDRESS_HOUSE_NO_AND_STREET',
                    candidate_to_node_relation=f'ADDRESS_HOUSE_NO_AND_STREET_{i}',
                    node_to_candidate_relation='ADDRESS_HOUSE_NO_AND_STREET_OF'
                )

        for i in range(0, 2):
            if row[f'predictions_primary_information_contact_information_addresses_{i}_country']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[f'predictions_primary_information_contact_information_addresses_{i}_country'],
                    node_type='ADDRESS_COUNTRY',
                    candidate_to_node_relation=f'ADDRESS_COUNTRY{i}',
                    node_to_candidate_relation='ADDRESS_COUNTRY_OF'
                )

        for i in range(0, 2):
            if row[f'predictions_primary_information_contact_information_addresses_{i}_locality']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[f'predictions_primary_information_contact_information_addresses_{i}_locality'],
                    node_type='ADDRESS_LOCALITY',
                    candidate_to_node_relation=f'ADDRESS_LOCALITY{i}',
                    node_to_candidate_relation='ADDRESS_LOCALITY_OF'
                )

        for i in range(0, 2):
            if row[f'predictions_primary_information_contact_information_addresses_{i}_postal_code']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[
                        f'predictions_primary_information_contact_information_addresses_{i}_postal_code'],
                    node_type='ADDRESS_POSTAL_CODE',
                    candidate_to_node_relation=f'ADDRESS_POSTAL_CODE{i}',
                    node_to_candidate_relation='ADDRESS_POSTAL_CODE_OF'
                )

        for i in range(0, 2):
            if row[f'predictions_primary_information_contact_information_addresses_{i}_region']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[f'predictions_primary_information_contact_information_addresses_{i}_region'],
                    node_type='ADDRESS_REGION',
                    candidate_to_node_relation=f'ADDRESS_REGION{i}',
                    node_to_candidate_relation='ADDRESS_REGION_OF'
                )

        for i in range(0, 4):
            if row[f'predictions_primary_information_contact_information_languages_known_{i}_actual_value']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[
                        f'predictions_primary_information_contact_information_languages_known_{i}_actual_value'],
                    node_type='LANGUAGE',
                    candidate_to_node_relation=f'LANGUAGE{i}',
                    node_to_candidate_relation='LANGUAGE_KNOWN_BY',
                    predicted_value=row[
                        f'predictions_primary_information_contact_information_languages_known_{i}_predicted_label']
                )

        for i in range(0, 54):
            if row[f'predictions_skills_hard_skills_{i}_actual_value']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[f'predictions_skills_hard_skills_{i}_actual_value'],
                    node_type='TECHNICAL SKILLS',
                    candidate_to_node_relation='HAS_SKILL',
                    node_to_candidate_relation='LEARNT_BY',
                    predicted_value=row[f'predictions_skills_hard_skills_{i}_predicted_label']
                )

        for i in range(0, 15):
            if row[f'predictions_skills_soft_skills_{i}_actual_value']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[f'predictions_skills_soft_skills_{i}_actual_value'],
                    node_type='PERSONAL SKILLS',
                    candidate_to_node_relation='HAS_SKILL',
                    node_to_candidate_relation='LEARNT_BY',
                    predicted_value=row[f'predictions_skills_soft_skills_{i}_predicted_label']
                )

        for i in range(0, 18):
            if row[f'predictions_work_experiences_{i}_designation']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[f'predictions_work_experiences_{i}_designation'],
                    node_type='PREVIOUS ROLE',
                    candidate_to_node_relation=f'PREVIOUS_ORGANIZATION_{i}_ROLE',
                    node_to_candidate_relation=f'PREVIOUS_ROLE'
                )

        for i in range(0, 18):
            if row[f'predictions_work_experiences_{i}_organization_organization_name']:
                graph = _add_node_and_edge(
                    graph=graph,
                    candidate_node=unique_id_node,
                    node_for_adding=row[f'predictions_work_experiences_{i}_organization_organization_name'],
                    node_type='PREVIOUS ORGANIZATION',
                    candidate_to_node_relation=f'PREVIOUS_ORGANIZATION_{i}',
                    node_to_candidate_relation=f'PREVIOUS_ORGANIZATION'
                )

    end_time = time.time()
    total_time = round(end_time - start_time, 3)
    print(f"Total time taken in seconds: {total_time}")
    # graph.add_edge("Node1", "Node2")
    # nx.draw(graph, with_labels=True)
    # plt.show()
    # print("graph nodes",list(graph.nodes))
    return graph


# def fetch_kg_from_zos(zg_id, config_id, tag_name):
#     zos_obj = ZosManager.get_object(constants.BUCKET_NAME, f"{zg_id}/{config_id}")
#     if zos_obj is not None:
#         raise Exception("no user exists")
#     if tag_name in zos_obj:
#         kg = zos_obj[tag_name][constants.KG]
#     else:
#         raise Exception("tag name deos not exists")
#
#     return kg

def draw_graph(graph, title):
    pos = nx.spring_layout(graph)  # Positions for all nodes
    nx.draw(graph, pos, with_labels=True, node_size=500, node_color="lightblue", font_size=10, font_weight="bold")
    plt.title(title)
    plt.show()
def load_graph_2():
    # def load_resumes():
    #     try:
    #         with open(
    #                 "/Users/rajes-19097/Documents/ZOHO/Gitlab2/multilingual_ver_2/CrmIntelligencePy/NumericAnalytics/src/SearchableResumeAppV3/result.json",
    #                 'r') as json_file:
    #             loaded_data = json.load(json_file)
    #         return loaded_data['predictions']
    #     except Exception as e:
    #         return []
    #
    # def convert_data_and_store_data(tag_name='test', str_json_data=None):
    #     if not str_json_data:
    #         str_json_data = load_resumes()
    #     data = StringIO(str_json_data)
    #
    #     df = pd.read_csv(data, sep=',')
    #     return df

    tag_name = "test"
    df = pd.read_csv(
        "/Users/rajes-19097/Documents/ZOHO/All_Git/Resume_App/NumericAnalytics/src/PurchaseAnalytics/reas/searchable_resume_app/utils_common/all_predictions.csv")
    graph = construct_graph(df.head(2))
    draw_graph(graph, "before")
    return graph


load_graph_2()


def fetch_graph_from_zos(zg_id, config_id, tag_name):
    kg = load_graph_2()
    file_path_exists = True
    # create file path
    # check
    # if file path exists
    # if not give custom response
    # fetch from zos
    # return kg and file_path_exists boolean
    return kg


SyntaxError: expected ':' (<ipython-input-6-12ebe3fbe304>, line 55)